In [1]:
# 引入模組
import pymysql
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import copy
import random

# 連接資料庫
conn = pymysql.connect("127.0.0.1", port=3306, user="root", passwd="",db="tourism1")
cursor = conn.cursor()

In [2]:
# 抓DB資料存成list或dict
cursor.execute("select * from attraction")
res = cursor.fetchall()
attraction = pd.DataFrame(list(res))
cursor.execute("select * from mrt")
res = cursor.fetchall()
mrt = pd.DataFrame(list(res))
cursor.execute("select * from arelated")
res = cursor.fetchall()
arelated = pd.DataFrame(list(res))
cursor.execute("select * from mrelated")
res = cursor.fetchall()
mrelated = pd.DataFrame(list(res))
cursor.execute("select * from tags")
res = cursor.fetchall()
tags = pd.DataFrame(list(res))

attraction.set_index(attraction[0], drop=True, inplace=True)
attraction = attraction.to_dict(orient='index')
mrt.set_index(mrt[0], drop=True, inplace=True)
mrt = mrt.to_dict(orient='index')
arelated = arelated.values
mrelated = mrelated.values
tags = tags.values

In [3]:
# 輸入景點，回傳所有相關景點
def findAllViewpoints(userInput):
    viewList = {}
    cursor.execute("select aId,aName from attraction where aName like '%" + userInput + "%'")
    res = cursor.fetchall()
    for (row1,row2) in res:
        viewList[row1] = row2
        
    return viewList

userInput = input()
findAllViewpoints(userInput)

{0: '鬼斧神宮',
 2: '臺灣富品',
 3: '千佛山福慧寺',
 4: '北天宮',
 5: '匠悅養身會館',
 6: '睿竹宮',
 7: '鴨拓草民宿',
 8: '田寮咖啡莊園 skyvillagecoffee',
 9: '何食代',
 10: '宗億精緻西點麵包',
 11: '帕莎蒂娜駁二倉庫餐廳',
 12: '益賢餐廳',
 13: '真天嚴(九天玄女娘娘)',
 14: '清心福全大樹中興店-珍珠奶茶手搖飲料專賣店',
 15: '美濃波波洛',
 16: '益賢山莊',
 17: '常美冰店魔法阿嬤的新家',
 18: '菸樓陶藝',
 19: '興達遠洋漁港',
 20: '蕃茄小姐與蘿蔔先生',
 21: "Jack's Gourmet Kitchen?傑克廚房",
 22: '九芎林里社真官伯公',
 23: '山水炎親子露營區',
 24: '山水炎餐廳',
 25: '孩餃一號水餃王(燕巢店)',
 26: '迪亞夢朵烘焙',
 27: '崗山之眼',
 28: '提摩希',
 29: '普普的風',
 30: '無老鍋(苓雅店)',
 31: '豐進成商店',
 32: '書香庭',
 33: '靈興殿',
 34: '大安禪寺',
 35: '丹丹漢堡 仁武店',
 36: '永新灣',
 37: '石二鍋_高雄鳳山店',
 38: '禾光牧場-羊咩咩的家',
 39: '伏流水模場公園',
 40: '林園清水寺',
 41: '真我山',
 42: '義大天悅飯店義大百匯自助餐廳',
 43: '萬應公',
 44: '冠中捲圓',
 45: '港埔江夏黃氏古厝',
 46: '愛麗絲居家生活館',
 47: '興田國小',
 48: '龍鳳山無極慈聖宮',
 49: '九號水門抽水站',
 50: '正東雜貨店',
 51: '玉清府',
 52: '汕尾北極殿',
 53: '高雄商旅URBANHOTEL33',
 54: '極鮮火鍋 - 澄清店',
 55: '犒吆包',
 56: '蓮溪埤',
 57: '雙溪樹木園',
 58: '保安金山寺',
 59: '映湖玥',
 60: '高雄 NL概念商旅',
 61: '異域農場',
 62: '墩仔腳',
 63: '九番埤溼地公園',
 64: '十八羅

In [5]:
# 經緯度計算兩點間距離
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r

# 定位過後回傳最近景點
def findNearestViewpoint(lng, lat):
    minDist = 10000
    minId = 1
    for i in attraction:
        d = haversine(lng, lat, attraction[i][6], attraction[i][5])
        if d < minDist:
            minDist = d
            minId = i
    return minId

lng = float(input("lng: "))
lat = float(input("lat: "))
nearId = findNearestViewpoint(lng, lat)
print(nearId, attraction[nearId][1])

lng: 126
lat: 22
710 曜專賣壽司


In [323]:
# 可能一：文字關聯
def text(view, attraction, arelated, aList, aWeight, isTag):
    b = 0
    # 非第一個景點，可能需要關聯
    if len(view) > 1: 
        tmp = []
        for i in arelated:
            if int(i[2]) == view[len(view)-1]:
                tmp.append(i[1])
        for i in arelated:
            if (int(i[2]) == view[len(view)-2]) and (i[1] in tmp) and (int(i[3]) in isTag) and (int(i[3]) not in allList) and (int(i[3]) not in view):
                aList[int(i[3])] = i[6]
                b = 1
    if b == 0:
        for i in arelated:
            if (int(i[2]) == view[len(view)-1]) and (int(i[3]) in isTag) and (int(i[3]) not in allList) and (int(i[3]) not in view):
                aList[int(i[3])] = i[6]
    
    # 計算view[0]跟aList景點的距離
    for i in aList:
        dist = haversine(attraction[view[len(view)-1]][6], attraction[view[len(view)-1]][5], attraction[i][6], attraction[i][5])
        if dist != 0:
            aList[i] = (1/dist) * aList[i] * aWeight * attraction[i][9]
        
# 可能二：距離最近捷運站附近之景點
def nearby(view, attraction, aList, dWeight, isTag, mList):
    nearMrt = attraction[view[len(view)-1]][7]
    mList.append(nearMrt)
    for i in attraction:
        if (attraction[i][7] == nearMrt) and (i not in allList) and (i not in view)  and (i in isTag):
            dist = haversine(attraction[view[len(view)-1]][6], attraction[view[len(view)-1]][5], attraction[i][6], attraction[i][5])
            if dist != 0:
                if i in aList:
                    aList[i] = aList[i] + (1/dist) * dWeight * attraction[i][9]
                else:
                    aList[i] = (1/dist) * dWeight * attraction[i][9]
                
# 可能三：捷運關聯
def data(view, attraction, mrelated, aList, mWeight, isTag, mList, isMrt):
    b = 0
    nextMrt = {}
    # 前一個景點來自可能三，可能需要關聯
    if len(mList) > 1: 
        tmp = []
        for i in mrelated:
            if i[2] == mList[len(mList)-1]:
                tmp.append(i[1])
        for i in mrelated:
            if (i[2] == mList[0]) and (i[1] in tmp) and (i[3] not in mList):
                nextMrt[int(i[3])] = i[6]
                b = 1
    if b == 0:
        for i in mrelated:
            if (i[2] == mList[len(mList)-1]) and (i[3] not in mList):
                nextMrt[int(i[3])] = i[6]
    
    # 關聯後捷運站的附近景點
    for i in nextMrt:
        for j in attraction:
            if (attraction[j][7] == i) and (j not in allList) and (j not in view) and (j in isTag):
                isMrt[j] = i
                # 景點間的距離用經緯度計算
                dist = haversine(attraction[view[len(view)-1]][6], attraction[view[len(view)-1]][5], attraction[j][6], attraction[j][5])
                # 距離用景點跟捷運站間的距離計算
                # dist = attraction[j][8]
                if dist != 0:
                    if j in aList:
                        aList[j] = aList[j] + (1/dist) * nextMrt[i] * mWeight * attraction[j][9]
                    else:
                        aList[j] = (1/dist) * nextMrt[i] * mWeight * attraction[j][9]
    
# 最初路線推薦
start = 1260                        #應該是吃景點id
inputTags = "自然風景 歷史古蹟"

# 權重宣告
aWeight = 0.2
dWeight = 0.5
mWeight = 0.3

# 找出符合tag的景點
result = [[] for i in range(5)]      #紀錄最後路線結果
resId = [[] for i in range(5)]       #紀錄最後路線結果id
allList = []                         #記錄景點id（判斷用）
tag = [n for n in inputTags.split()] #記錄所有tag
isTag = []                           #紀錄符合tag的景點們
mList = []                           #紀錄計算過程中有用到的捷運站
copyOfmList = []                     #紀錄最初捷運站（跑後面的路線會用到）
view = []                            #暫存上一個點
resList = []                         #暫存排序後的結果
aList = {}                           #暫存景點結果（景點id+加權後分數）
isMrt = {}                           #暫存如果是來自可能三的結果（會影響到mList）

for i in range(5):                   #第一層景點（使用者輸入的）
    result[i].append(attraction[start][1])
    resId[i].append(start)
    allList.append(start)
if len(tag) != 0:                    #有輸入tag
    for i in tags:
        if i[2] in tag:
            isTag.append(i[1])
else:                                #沒有指定tag，故所有景點都會考慮
    for i in attraction:
        isTag.append(i)

# 計算每條路線的第二個點
view.append(start)

text(view, attraction, arelated, aList, aWeight, isTag)
nearby(view, attraction, aList, dWeight, isTag, mList)
data(view, attraction, mrelated, aList, mWeight, isTag, mList, isMrt)
resList = sorted(aList.items(), key=lambda item: item[1], reverse=True)
copyOfmList = copy.copy(mList)

for i in range(5):
    if len(resList) > i:
        result[i].append(attraction[resList[i][0]][1])
        resId[i].append(resList[i][0])
        allList.append(resList[i][0])
    
    # 計算每條路線的第三個點
    view = []
    aList = {}
    view.append(start)
    if len(resList) > i:
        view.append(resList[i][0])
        mList = copy.copy(copyOfmList)
        if resList[i][0] in isMrt:
            mList.append(isMrt[resList[i][0]])

        text(view, attraction, arelated, aList, aWeight, isTag)
        nearby(view, attraction, aList, dWeight, isTag, mList)
        data(view, attraction, mrelated, aList, mWeight, isTag, mList, isMrt)
    
    if len(aList) != 0:
        maxScore = max(aList, key=aList.get)
        result[i].append(attraction[maxScore][1])
        resId[i].append(maxScore)
        allList.append(maxScore)
    if len(result[i]) > 1:
        print(result[i])
    else:
        print("查無景點路線")

['西子灣風景區', '打狗英國領事館文化園區山上官邸', '柴山']
['西子灣風景區', '柴山', '旗津海岸公園']
['西子灣風景區', '旗津天后宮', '山海莊海產土雞城']
['西子灣風景區', '旗津海岸公園', '斗六冰城']
['西子灣風景區', '山海莊海產土雞城', '海韻戀歌 Ocean Love Song']


In [330]:
# 更換中間景點
def changeTheSecond(resId, index, view, aList, maxDist):
    # 可能一（文字關聯）
    tmp = []
    tmp1 = []
    for i in arelated:
        if int(i[2]) == view[0]:
            tmp1.append(i[1])
        if (int(i[3]) == view[1]) and (i[1] in tmp1):
            tmp.append(i[1])
    for i in arelated:
        if (i[1] in tmp) and (int(i[2]) in isTag) and (int(i[2]) not in view):
            aList[int(i[2])] = i[6]
    
    for i in aList:
        dist = haversine(attraction[view[0]][6], attraction[view[0]][5], attraction[i][6], attraction[i][5])
        if (dist != 0) and (dist <= maxDist):
            aList[i] = (1/dist) * aList[i] * aWeight * attraction[i][9]
    
    # 可能二（距離最近捷運站附近之景點）
    nearMrt1 = attraction[view[0]][7]
    for i in attraction:
        if (attraction[i][7] == nearMrt1) and (i not in view)  and (i in isTag):
            dist = haversine(attraction[view[0]][6], attraction[view[0]][5], attraction[i][6], attraction[i][5])
            if (dist != 0) and (dist <= maxDist):
                if i in aList:
                    aList[i] = aList[i] + (1/dist) * dWeight * attraction[i][9]
                else:
                    aList[i] = (1/dist) * dWeight * attraction[i][9]
                   
    # 可能三（捷運關聯）
    nearMrt2 = attraction[view[1]][7]
    tmp2 = []
    tmp3 = [] 
    nextMrt = {}
    for i in mrelated:
        if i[2] == nearMrt1:
            tmp2.append(i[1])
        if (i[3] == nearMrt2) and (i[1] in tmp2):
            tmp3.append(i[1])
    for i in mrelated:
        if (i[1] in tmp3) and (i[2] != nearMrt1):
            nextMrt[int(i[2])] = i[6]
    
    for i in nextMrt:
        for j in attraction:
            if (attraction[j][7] == i) and (j not in view) and (j in isTag):
                dist = haversine(attraction[view[0]][6], attraction[view[0]][5], attraction[j][6], attraction[j][5])
                if (dist != 0) and (dist <= maxDist):
                    if j in aList:
                        aList[j] = aList[j] + (1/dist) * nextMrt[i] * mWeight * attraction[j][9]
                    else:
                        aList[j] = (1/dist) * nextMrt[i] * mWeight * attraction[j][9]
    
    # 選擇更換景點
    if resId[index][1] in aList:
        del aList[resId[index][1]]
    if len(aList) == 0:
        print("沒有更好的景點了！")
        print(result[index])
    else:
        aNew, scoreNew = random.choice(list(aList.items()))
        resId[index][1] = aNew                    
        result[index][1] = attraction[aNew][1]
        print(attraction[aNew][1])
        print(result[index])

# 更換最末端景點
def changeTheLast(resId, index, view, aList, mList):
    text(view, attraction, arelated, aList, aWeight, isTag)
    nearby(view, attraction, aList, dWeight, isTag, mList)
    data(view, attraction, mrelated, aList, mWeight, isTag, mList, isMrt)
    
    if resId[index][2] in aList:
        del aList[resId[index][2]]
    if len(aList) == 0:
        print("沒有更好的景點了！")
        print(result[index])
    else:
        # 隨機取本次計算的值，只會跟上次不一樣，多次嘗試下來可能換到重覆景點
        aNew, scoreNew = random.choice(list(aList.items()))
        resId[index][2] = aNew                  #紀錄新景點id                    
        result[index][2] = attraction[aNew][1]  #紀錄新景點名稱
        print(attraction[aNew][1])              #印出更新後的景點
        print(result[index])                    #印出更新後的該條路線
    
# 替換景點
index = 0       #以result第1筆作為更改範例
change = 2      #1->替換第二個點；2->替換第三個點

view = []
mList = []
aList = {}
view.append(resId[index][0])
if change == 1:
    view.append(resId[index][2])
    maxDist = haversine(attraction[view[0]][6], attraction[view[0]][5], attraction[view[1]][6], attraction[view[1]][5])
    changeTheSecond(resId, index, view, aList, maxDist)
elif change == 2:
    view.append(resId[index][1])
    changeTheLast(resId, index, view, aList, mList)

旗津星空隧道
['西子灣風景區', '旗後砲台', '旗津星空隧道']


In [334]:
# 選定路線後新增景點
index = 0       #以result第一筆作為更改範例
mList = []
aList = {}

text(resId[index], attraction, arelated, aList, aWeight, isTag)
nearby(resId[index], attraction, aList, dWeight, isTag, mList)
data(resId[index], attraction, mrelated, aList, mWeight, isTag, mList, isMrt)

if len(aList) == 0:
    print("沒有更好的景點了！")
    print(result[index])
elif len(resId[index]) < 6:
    maxScore = max(aList, key=aList.get)
    result[index].append(attraction[maxScore][1])
    resId[index].append(maxScore)
    print(attraction[maxScore][1])  #輸出本次新增景點
    print(result[index])            #輸出最終路線
else:
    print("已達景點數上限！")
    print(result[index])

已達景點數上限！
['西子灣風景區', '旗後砲台', '旗津星空隧道', '胡椒手工魚麵店', '旗津道沙灘酒店 In Young Hotel', '旗津海水浴場']
